In [ ]:
#import Libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  StandardScaler
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor as xg

# Let's Read DataSet !

In [ ]:
data=pd.read_csv("train.csv")
da=data

## Let's Analyse data!

In [ ]:
# let's show dataset
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Let's check if there is null values in dataset!

In [ ]:
data.isnull().sum().sum()

6965

## Let's check if any columns is not important for model !

In [ ]:
data.drop(axis=1,columns=['Id'],inplace=True)
data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
clear=[]
for i in data.columns :
  if (data[i].isna().sum()>600):
    clear.append(i)
    data.drop(i,axis=1,inplace=True);

data.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [ ]:
# Let's drop rows which have null values!
data.dropna(axis=0,inplace=True)
columns_old=list(data.columns)


In [ ]:
data.shape

(1094, 75)

### Let's check if columns's data has wrong data types !

# Let's handle with unnumaric data!

In [ ]:
# Let's drop unnumric columns!
un=[]
for i in data.columns :
    if data[i].dtype=='object':
        un.append(i)

data =pd.get_dummies(data, columns = un)
data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
data.isnull().sum().sum()

0

In [ ]:
data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,...,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000,1094.000000
mean,56.128885,70.759598,10132.346435,6.247715,5.575868,1972.412249,1985.915905,109.855576,448.191956,45.252285,...,0.003656,0.106033,0.000914,0.848263,0.063985,0.000914,0.005484,0.016453,0.804388,0.108775
std,41.976345,24.508859,8212.249621,1.366797,1.066500,31.189752,20.930772,190.667459,468.728095,159.075003,...,0.060384,0.308021,0.030234,0.358930,0.244839,0.030234,0.073888,0.127269,0.396853,0.311499
min,20.000000,21.000000,1300.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,60.000000,7606.750000,5.000000,5.000000,1953.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,50.000000,70.000000,9444.500000,6.000000,5.000000,1975.000000,1995.000000,0.000000,384.500000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,70.000000,80.000000,11387.250000,7.000000,6.000000,2003.000000,2005.000000,171.750000,712.750000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Let's Split Data to features and target columns!

In [ ]:
da=data

In [ ]:
X_da=da.iloc[:,:].drop('SalePrice',axis=1).values
y=da.iloc[:,-1].values
y_da=y
print(data.shape)
da

(1094, 260)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
# da.drop(columns=['SalePrice', 'Condition2_RRNn', 'HouseStyle_2.5Fin', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 'Exterior1st_Stone', 'Exterior2nd_Other', 'Heating_OthW', 'Electrical_Mix', 'GarageQual_Ex'],axis=1,inplace=True)

In [ ]:
# from sklearn.feature_selection import f_regression
# #define feature selection
# fs = SelectKBest(score_func=f_regression, k=259)
# # apply feature selection
# X_selected = fs.fit_transform(X_da, y_da)

# #fs.get_support(indices=True)

# X_selected

In [ ]:
# cols=[]
# for i in fs.get_support(indices=True):
#     cols.append(data.iloc[:,i].name)
# print(cols)

In [ ]:
# print(data.columns.drop('SalePrice'))
# for col in data.columns:
#     fig = plt.figure(figsize=(2,2))
#     plt.title(f"{col}")
#     plt.scatter(data[col],data['SalePrice'])
#     plt.show()

### Let's check outliears in some the best features!

In [ ]:
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(251)
# plt.boxplot(data['OverallQual'],labels=' ');
# plt.xlabel('OverallQual')
# plt.subplot(252)
# plt.boxplot(data['YearBuilt'],labels=' ');
# plt.xlabel('YearBuilt')
# plt.subplot(253)
# plt.boxplot(data['YearRemodAdd'],labels=' ');
# plt.xlabel('YearRemodAdd')
# plt.subplot(254)
# plt.boxplot(data['TotalBsmtSF'],labels=' ');
# plt.xlabel('TotalBsmtSF')
# plt.subplot(255)
# plt.boxplot(data['1stFlrSF'],labels=' ');
# plt.xlabel('1stFlrSF');

# for i in
#     plt.subplot(255)
#     plt.boxplot(data['1stFlrSF'],labels=' ');
#     plt.xlabel('1stFlrSF');

In [ ]:
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(251)
# plt.boxplot(data['GrLivArea'],labels=' ');
# plt.xlabel('GrLivArea')
# plt.subplot(252)
# plt.boxplot(data['FullBath'],labels=' ');
# plt.xlabel('FullBath')
# plt.subplot(253)
# plt.boxplot(data['TotRmsAbvGrd'],labels=' ');
# plt.xlabel('TotRmsAbvGrd')
# plt.subplot(254)
# plt.boxplot(data['GarageCars'],labels=' ');
# plt.xlabel('GarageCars')
# plt.subplot(255)
# plt.boxplot(data['GarageArea'],labels=' ');
# plt.xlabel('GarageArea');

### So there is some features like 'YearBuilt ' , '1stFlrSF ','TotalBsmtSF ','GrLivArea' has outliears Let's regularize them!

In [ ]:
# # Let's try the log function!
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(241)
# plt.boxplot(np.log(data['YearBuilt']),labels=' ');
# plt.xlabel('YearBuilt')
# plt.subplot(242)
# plt.boxplot(np.log(data['TotalBsmtSF']),labels=' ');
# plt.xlabel('TotalBsmtSF')
# plt.subplot(243)
# plt.boxplot(np.log(data['1stFlrSF']),labels=' ');
# plt.xlabel('1stFlrSF');
# plt.subplot(244)
# plt.boxplot(np.log(data['GrLivArea']),labels=' ');
# plt.xlabel('GrLivArea');
# # ------------"after log function"

In [ ]:
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(221)
# plt.boxplot(np.log(data['GrLivArea']),labels=' ');
# plt.xlabel('GrLivArea')
# plt.subplot(222)
# plt.boxplot(np.log(data['GarageArea']),labels=' ');
# plt.xlabel('GarageArea');
# # ------------"after log function"

### After taken log for the most columns that have a lot of outliears ,I found out log doesn't affect Significantly in this columns unless 'GarageArea' column .

### So let's use another technique to handle outliears in this columns!

In [ ]:
data.shape

(1094, 260)

In [ ]:
# data=data[cols]
# data.shape

In [ ]:
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
data.dtypes

MSSubClass                 int64
LotFrontage              float64
LotArea                    int64
OverallQual                int64
OverallCond                int64
                          ...   
SaleCondition_AdjLand      uint8
SaleCondition_Alloca       uint8
SaleCondition_Family       uint8
SaleCondition_Normal       uint8
SaleCondition_Partial      uint8
Length: 260, dtype: object

In [ ]:
#Let's try get ride of outliers!
for x in data.columns:
   if data[x].dtype=='int64' or data[x].dtype=='float64' :
        q75,q25 = np.percentile(data.loc[:,x],[75,25])
        intr_qr = q75-q25

        max = q75+(1.5*intr_qr)
        min = q25-(1.5*intr_qr)

        data.loc[data[x] > max,x] = max
        data.loc[data[x] < min,x] = min


In [ ]:
data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5.0,2003,2003,196.0,706.0,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,7.5,1976,1976,0.0,978.0,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5.0,2001,2002,162.0,486.0,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5.0,1915,1970,0.0,216.0,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5.0,2000,2000,350.0,655.0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5.0,1999,2000,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6.0,1978,1988,119.0,790.0,0,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,7.5,1941,2006,0.0,275.0,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6.0,1950,1996,0.0,49.0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
# data['SalePrice'].isna().sum()

In [ ]:
print(data.shape)

(1094, 260)


In [ ]:
# # Let's drop outliers!

# data.dropna(inplace=True,axis=0)

In [ ]:
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(251)
# plt.boxplot(data['OverallQual'],labels=' ');
# plt.xlabel('OverallQual')
# plt.subplot(252)
# plt.boxplot(data['YearBuilt'],labels=' ');
# plt.xlabel('YearBuilt')
# plt.subplot(253)
# plt.boxplot(data['YearRemodAdd'],labels=' ');
# plt.xlabel('YearRemodAdd')
# plt.subplot(254)
# plt.boxplot(data['TotalBsmtSF'],labels=' ');
# plt.xlabel('TotalBsmtSF')
# plt.subplot(255)
# plt.boxplot(data['1stFlrSF'],labels=' ');
# plt.xlabel('1stFlrSF');
# #----------------"after geting rid of outliers"

In [ ]:
# plt.rcParams["figure.figsize"] = [14.50, 7.50]
# plt.subplot(241)
# plt.boxplot(np.log(data['YearBuilt']),labels=' ');
# plt.xlabel('YearBuilt')
# plt.subplot(242)
# plt.boxplot(np.log(data['TotalBsmtSF']),labels=' ');
# plt.xlabel('TotalBsmtSF')
# plt.subplot(243)
# plt.boxplot(np.log(data['1stFlrSF']),labels=' ');
# plt.xlabel('1stFlrSF');
# plt.subplot(244)
# plt.boxplot(np.log(data['GrLivArea']),labels=' ');
# plt.xlabel('GrLivArea');
# #----------------"after geting rid of outliers"

In [ ]:
#let's check if there are outliers!
data.isna().sum().sum()

0

In [ ]:
# # Let's convert datatypes for columns to can pass them to regression model!
# for x in data.columns:
#     data[x]=data[x].astype('int')

In [ ]:
# data.dtypes

#### Let's Split Data to features and target columns!

In [ ]:
# X=data[cols].values

# y=data['SalePrice']
# X

In [ ]:
# # Scaling data
# Scalar=StandardScaler()
# X=Scalar.fit_transform(X)

In [ ]:
# print(X.shape,"\t",y.shape)

### Let's Split data to train and test data!

In [ ]:
data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5.0,2003,2003,196.0,706.0,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,7.5,1976,1976,0.0,978.0,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5.0,2001,2002,162.0,486.0,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5.0,1915,1970,0.0,216.0,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5.0,2000,2000,350.0,655.0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5.0,1999,2000,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6.0,1978,1988,119.0,790.0,0,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,7.5,1941,2006,0.0,275.0,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6.0,1950,1996,0.0,49.0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data.drop('SalePrice',axis=1),data['SalePrice'],test_size=.3,random_state=0)

print(X_train,"\n\n--------",X_test,"\n\n--------",y_train,"\n\n--------",y_test)

      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
248           60         72.0    11317            7          5.0       2003   
563           50         66.0    17058            6          7.0       1918   
1366          60         68.0     9179            7          5.0       1999   
10            20         70.0    11200            5          5.0       1965   
1139          30         98.0     8731            5          5.0       1920   
...          ...          ...      ...          ...          ...        ...   
1384          50         60.0     9060            6          5.0       1939   
1016          20         73.0    11883            7          5.0       1996   
1112          20         73.0     7100            5          7.0       1957   
739           60         65.0     9313            7          5.0       2004   
899           20         65.0     6993            5          7.0       1961   

      YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFin

In [ ]:
da

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5.0,2003,2003,196.0,706.0,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,7.5,1976,1976,0.0,978.0,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5.0,2001,2002,162.0,486.0,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5.0,1915,1970,0.0,216.0,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5.0,2000,2000,350.0,655.0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5.0,1999,2000,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6.0,1978,1988,119.0,790.0,0,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,7.5,1941,2006,0.0,275.0,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6.0,1950,1996,0.0,49.0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
X_datrain,X_datest,y_datrain,y_datest=train_test_split(X_da,y_da,test_size=.3,random_state=0)

#print(X_train,"\n\n--------",X_test,"\n\n--------",y_train,"\n\n--------",y_test)

# Let's create Regression model!

In [ ]:
# linear regression
from sklearn.linear_model import LinearRegression
regression=LinearRegression()
regression.fit(X_train,y_train)
regression.coef_

array([ 3.71264141e+01,  1.05677860e+02,  1.11263051e+00,  7.35648107e+03,
        6.27336269e+03,  1.38229084e+02,  1.39212701e+02,  1.92210121e+01,
       -7.54109959e+00, -3.11055487e-06, -1.88291252e+01,  4.01536837e+01,
       -9.14142516e+00,  1.07123482e+01,  2.21551909e-06,  5.18468200e+01,
        4.72794170e+03,  3.38887276e-07,  2.57683755e+03,  2.39041263e+03,
       -1.47835738e+02, -2.41416274e-08, -6.36408280e+02,  1.71146559e+03,
       -3.06370186e+00,  4.22513927e+03,  2.32900750e+01,  1.08196096e+01,
        4.81731888e+01, -3.40151018e-09,  1.49939297e-08,  8.05630407e-09,
       -7.50515028e-09,  1.82262738e-09,  8.72087358e+01, -1.29487004e+02,
       -2.78974458e+04,  1.71314471e+04,  1.03368737e+04,  3.27032982e+03,
       -2.84120470e+03, -1.43016822e+04,  1.43016822e+04, -8.61441640e+03,
       -6.87273727e+03,  2.14097492e+04, -5.92259551e+03, -1.66032824e+03,
       -4.22595786e+03,  1.19968300e+04, -6.11054388e+03,  6.09361450e-11,
        9.50852932e+02,  

In [ ]:
# Let's Test the model!
y_pred=regression.predict(X_test)

# Let's measure Error and Score of model !

In [ ]:
from sklearn.metrics import mean_absolute_error
Error=mean_absolute_error(y_test,y_pred)
Error

15878.09495789442

In [ ]:
from sklearn.metrics import mean_squared_error
Error2=mean_squared_error(y_test,y_pred)
Error2

574344291.1392418

In [ ]:
# score model in fitting data
from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)
score

0.8710310895850932

In [ ]:
#print(y_test,"\t",y_pred)

# Let's create GradientBoosting model!

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
model=GradientBoostingRegressor().fit(X_datrain,y_datrain)
#params={'n_estimators':range(1,200)}
#grid=GridSearchCV(estimator=model,cv=2,param_grid=params,scoring='neg_mean_squared_error')
#grid.fit(X,Y)
#print("The best estimator returned by GridSearch CV is:",grid.best_estimator_)
#Output
#The best estimator returned by GridSearch CV is:  GradientBoostingRegressor(n_estimators=19)
#GB=grid.best_estimator_
#GB.fit(X,Y)


In [ ]:
Y_predict=model.predict(X_datest)
error= mean_squared_error(y_datest,Y_predict)
error

7.385546235197355e-11

In [ ]:
from sklearn.metrics import r2_score
score=r2_score(y_datest,Y_predict)
score

0.9999999992931752

In [ ]:
xgmodel =xg(reg_alpha=10).fit(X_datrain,y_datrain)
Y_predict=xgmodel.predict(X_datest)
from sklearn.metrics import r2_score
score=r2_score(y_datest,Y_predict)
score

0.9804598082060976

## Let's Read test file!

In [ ]:
test=pd.read_csv("test.csv")
test.shape

(1459, 80)

In [ ]:
# slice id column from test file
submission_id=test.loc[:,'Id'].transpose()
submission_id

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [ ]:
# columns_old.remove('SalePrice')
for i in test.columns :
  for j in clear :
    if i == j :
      test.drop(i,inplace=True,axis=1)
test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPo

In [ ]:
test.shape

(1459, 75)

In [ ]:
# # for i in test.columns:
# #   print( i)
# test = test[columns_old]

In [ ]:
# Let's drop unnumric columns!
un=[]
for i in test.columns :
    if test[i].dtype=='object':
        un.append(i)

test =pd.get_dummies(test, columns = un)
test

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,0,0,1,1,0,0,0,0,0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
# da.drop(columns=['SalePrice', 'Condition2_RRNn', 'HouseStyle_2.5Fin', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 'Exterior1st_Stone', 'Exterior2nd_Other', 'Heating_OthW', 'Electrical_Mix', 'GarageQual_Ex'],axis=1,inplace=True)

In [ ]:
# The columns will use in prediction.
x_test=test[da.columns]

print(x_test.shape)


KeyError: ignored

In [ ]:
x_test.head()

In [ ]:
#Let's check if there are any missing value so can pass them to regression model!
x_test.isnull().sum().sum()

In [ ]:
# to fill na with 0
x_test.fillna(0,inplace=True)

In [ ]:
x_test.isnull().sum().sum()

In [ ]:
# Scaling data
# Scalar=StandardScaler()
# x_test=Scalar.fit_transform(x_test)

# -------Scaling data made a wrong prediections which made predictions in negative values

# x_test

# Let's predict price from the test file!

In [ ]:
submission_pred=xgmodel.predict(x_test)
submission_pred.shape

In [ ]:
submission=pd.DataFrame(data=submission_id)

submission1=pd.DataFrame(data=submission_pred)

#submission1.astype(int).to_csv('submission.csv', index=False)
submission.insert(1, column = "SalePrice", value =submission1)

#Let's save the result to submission file.
submission.astype(int).to_csv('submission.csv', index=False)

In [ ]:
submission.head(-10)

In [ ]:
r=pd.read_csv('submission.csv')
r.head(-10)

In [ ]:
# # Rmoving submission file
# import os

# os.remove('submission.csv')